In [2]:
import json
import os
from pathlib import Path
import uuid
import random
import time
from datetime import datetime, timezone

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType
import pyspark.sql.functions as F

PROJECT_PATH = Path.cwd().parent
DATA_DIR = '.data'
DATA_PATH = PROJECT_PATH / DATA_DIR
DATA_PATH.mkdir(exist_ok=True)

output_folder = str(DATA_PATH)
checkpoint_path = str(Path.cwd() / 'checkpoint')

file_schema = StructType() \
    .add('id', StringType()) \
    .add('temperature', DoubleType()) \
    .add('timestamp', TimestampType())

schema_name = 'dp700_e011'
table_name = 'temperature_stream'

spark = SparkSession.builder.appName('test').master('local[*]').getOrCreate()


In [3]:
spark.sql(f'CREATE SCHEMA IF NOT EXISTS {schema_name}')


DataFrame[]

In [4]:
raw_stream_df = spark.readStream \
    .schema(file_schema) \
    .option('maxFilesPerTrigger', 1) \
    .json(output_folder)

transformed_stream_df = raw_stream_df \
    .withColumn('processed_timestamp',
        F.current_timestamp())

delta_stream = transformed_stream_df.writeStream \
    .format('delta') \
    .outputMode('append') \
    .option('checkpointLocation', checkpoint_path) \
    .start(f'Tables/{schema_name}/{table_name}')

Py4JJavaError: An error occurred while calling o51.start.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:370)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:233)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:569)
	at py4j.

In [ ]:
df = spark.sql(f'SELECT * FROM {schema_name}.{table_name}')

In [ ]:
spark.stop()